In [2]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import matplotlib.image as mimg
from tqdm.auto import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

from pathlib import Path
train_dir=Path('dermnet/train/')
test_dir=Path('dermnet/test')
data_dir=Path('dermnet')
train_dir,test_dir

from torchvision.transforms import v2

from PIL import Image

data_transform=v2.Compose([
  v2.Resize((256,256)),
  v2.RandomCrop((256,256)),
  v2.RandomHorizontalFlip(),
  v2.ToImage(),
  v2.ToDtype(torch.float32,scale=True)
])

from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset,DataLoader

train_data=ImageFolder(train_dir,transform=data_transform,
                       target_transform=None,
                       )
test_data=ImageFolder(test_dir,transform=data_transform)


train_dataloader = DataLoader(train_data, batch_size=320,num_workers=os.cpu_count(), shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=320,num_workers=os.cpu_count(), shuffle=False)

train_data,test_data,len(train_dataloader),len(test_dataloader)

import torchvision.models as models
from torchinfo import summary
weights=models.MobileNet_V3_Large_Weights.DEFAULT
model = models.mobilenet_v3_large(weights=weights).to(device)

summary(model=model,
input_size=( 1,3, 256, 256),
col_names=['input_size', 'output_size', 'num_params','trainable'],
col_width=20,
row_settings=['var_names'])

model.classifier=nn.Sequential(
nn.Dropout(p=0.5,inplace=True),
nn.Linear(in_features=960,out_features=640),
nn.Dropout(p=0.2,inplace=True),
nn.LeakyReLU(),
nn.Linear(640,300),
nn.Dropout(p=0.2,inplace=True),
nn.Linear(300,len(train_data.classes)))
for param in model.features.parameters():
    param.requires_grad=False
    
summary(
    model=model,
    input_size=(1, 3, 256, 256),
    col_names=['input_size', 'output_size', 'num_params', 'trainable'],
    col_width=20,
    row_settings=['var_names']
)

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),0.001)
def acc(y_preds,y_true):
    correct=(y_preds==y_true).sum().item()
    
    acc=correct*100/len(y_preds)
    return acc

epochs =1

train_loss=[]
test_loss=[]
torch.manual_seed(42)
for epoch in tqdm(range(epochs)):
    model.train()


    los=0
    train_acc=0
    test_acc=0
    print(f'Epoch: {epoch}')
    print(f'------------------')
    for i,data in enumerate(train_dataloader):
        
        img,label=data[0],data[1]
        img,label=img.to(device),label.to(device)

        y_logits=model(img)
        loss=loss_fn(y_logits,label.type(torch.long))
        los+=loss
        y_preds=torch.argmax(torch.softmax(y_logits,1),1)
        train_acc+=acc(y_preds,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        if i%10==0:
            if len(label)==320:
                print(f'Looked at {(i+1)*len(label)}/{len(train_data)}...')
            else:
                print(f'Looked at {len(train_data)}/{len(train_data)}...')
    train_acc/=len(train_dataloader)
    los/=len(train_dataloader)
    train_loss.append(los.detach().cpu)

    for img,label in iter(test_dataloader):
        img,label=img.to(device),label.to(device)
        test_logits=model(img)
        test_preds=torch.argmax(torch.softmax(test_logits,1),1)
        test_acc+=acc(test_preds,label)
    test_loss.append(loss_fn(test_logits,label.type(torch.long)).detach().cpu())
    test_acc/=len(test_dataloader)
    print(f'Train_Loss:{los} | Train_accuracy:{train_acc}% | Test_accuracy:{test_acc}%')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 0
------------------
Looked at 320/15557...
Looked at 3520/15557...
Looked at 6720/15557...
Looked at 9920/15557...
Looked at 13120/15557...
Train_Loss:2.5156004428863525 | Train_accuracy:26.557482129907804% | Test_accuracy:33.09235517568851%


In [3]:
torch.save(model, "skin_disease.pt")  

In [4]:
model = torch.load("skin_disease.pt", map_location=torch.device('cpu'))

In [5]:
class_names=train_data.classes

In [6]:
class_names=['Type=Acne and Rosacea Photos 	Symptoms=Redness, pimples, blackheads, whiteheads, oily skin,Facial redness, flushing, pimples, bloodshot eyes	 Medications=Over-the-counter acne treatments, prescription medications, lifestyle changes,Topical medications, laser therapy, lifestyle changes',

 'Type=Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions 	Symptoms=No info	Medications=NO info',

 'Type=Atopic Dermatitis Photos 	Symptoms=No info	Medications=NO info',

 'Type=Bullous Disease Photos 	Symptoms=No info	Medications=NO info',

 'Type=Cellulitis Impetigo and other Bacterial Infections	 Symptoms=Red, swollen, painful skin	 Medications=Antibiotics',

 'Type=Eczema Photos 	Symptoms=Itchy, dry, red skin, scaly patches	 Medications=Moisturizers, topical corticosteroids, antihistamines',

 'Type=Exanthems and Drug Eruptions 	Symptoms=No info	Medications=NO info',

 'Type=Hair Loss Photos Alopecia and other Hair Diseases 	Symptoms=Hair loss in round patches	 Medications=Corticosteroids, immunotherapy, minoxidil',

 'Type=Herpes HPV and other STDs Photos 	Symptoms=Painful blisters	 Medications=Antiviral medications',

 'Type=Light Diseases and Disorders of Pigmentation	 Symptoms=No info	Medications=NO info',

 'Type=Lupus and other Connective Tissue diseases 	Symptoms=No info	Medications=NO info',

 'Type=Melanoma Skin Cancer Nevi and Moles 	Symptoms=Abnormal moles, changes in existing moles, dark or uneven color	 Medications=Surgery, immunotherapy, chemotherapy, radiation therapy',

 'Type=Nail Fungus and other Nail Disease 	Symptoms=No info	Medications=NO info',

 'Type=Poison Ivy Photos and other Contact Dermatitis 	Symptoms=No info	Medications=NO info',

 'Type=Psoriasis pictures Lichen Planus and related diseases	 Symptoms=Thick, scaly patches, red skin, itching	 Medications=Topical corticosteroids, light therapy, systemic medications',

 'Type=Scabies Lyme Disease and other Infestations and Bites	 Symptoms=No info	Medications=NO info',

 'Type=Seborrheic Keratoses and other Benign Tumors 	Symptoms=Dandruff, oily scales, red skin	 Medications=Over-the-counter dandruff shampoos, prescription corticosteroids',

 'Type=Systemic Disease 	Symptoms=No info	Medications=NO info',

 'Type=Tinea Ringworm Candidiasis and other Fungal Infections 	Symptoms=Itching, redness, scaling, blisters	 Medications=Antifungal creams, oral antifungals',

 'Type=Urticaria Hives 		Symptoms=Itchy red welts	 Medications=Antihistamines, corticosteroids',

 'Type=Vascular Tumors		Symptoms=No info	Medications=NO info ',

 'Type=Vasculitis Photos 	Symptoms=No info	Medications=NO info',

 'Type=Warts Molluscum and other Viral Infections 	Symptoms=Small, raised growths on skin		 Medications=Over-the-counter wart removal products, freezing, prescription medications']

In [7]:
class_names

['Type=Acne and Rosacea Photos \tSymptoms=Redness, pimples, blackheads, whiteheads, oily skin,Facial redness, flushing, pimples, bloodshot eyes\t Medications=Over-the-counter acne treatments, prescription medications, lifestyle changes,Topical medications, laser therapy, lifestyle changes',
 'Type=Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions \tSymptoms=No info\tMedications=NO info',
 'Type=Atopic Dermatitis Photos \tSymptoms=No info\tMedications=NO info',
 'Type=Bullous Disease Photos \tSymptoms=No info\tMedications=NO info',
 'Type=Cellulitis Impetigo and other Bacterial Infections\t Symptoms=Red, swollen, painful skin\t Medications=Antibiotics',
 'Type=Eczema Photos \tSymptoms=Itchy, dry, red skin, scaly patches\t Medications=Moisturizers, topical corticosteroids, antihistamines',
 'Type=Exanthems and Drug Eruptions \tSymptoms=No info\tMedications=NO info',
 'Type=Hair Loss Photos Alopecia and other Hair Diseases \tSymptoms=Hair loss in round patches\t Medicatio

In [8]:
import torch
import torchvision.transforms as transforms
import gradio as gr
import numpy as np

# Load the entire model (assuming it's saved as "skin_disease.pt")
model = torch.load("skin_disease.pt", map_location=torch.device('cpu'))  # Adjust device if needed

def classify_image(image):
    transform = v2.Compose([
      v2.Resize(256), v2.ToImage(),
      v2.ToDtype(torch.float32,scale=True)
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    with torch.inference_mode():
         pred_probs = torch.softmax(model(img), dim=1)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    return pred_labels_and_probs

# Create the Gradio interface
interface = gr.Interface(classify_image, "image", "label")

# Launch the interface
interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://456431b4c4d0037b1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
